# Downsample-timeseries-using-local-median

Suppose you have data sampled at a higher frequency, e.g. every second and you only need samples every minute.  If you simply use the [Pandas resample function](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html) your resampled signal will still be sensitive to any noise or other high frequency variations in the data that you are not interested in.

```python
samples_1min = x.resample(rule='1min').agg('first')
```

Alternatively, you could take an average of the values in each sample period.

```python
samples_1min = x.resample(rule='1min').mean()
```

But this does a lot of smoothing of the data and actually introduces a delay into any dynamics you are trying to identify.

Obviously, the best thing to do is to use an appropriate filter that takes into account the expected time-response of the system generating the data, but this requires a model of the dynamics.  When you are doing system identification, you don't have this model at the outset.  You could try fitting a model to all the data but that might be computationally expensive and unnecessary if the sampling frequency is very much faster than the dynamics you are trying to identify.

The technique I propose here is to take a short sample of values immediately before and after each sample time and take a median of these values to represent the estimated true value at the centre of each sample.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Prepare an artificial time series with a sample period of
# 1 second to represent the signal data
index = pd.date_range('2021-12-01 06:00:17', '2021-12-01 06:02:57', 
                      freq='1s', name='t')
data = {'x': range(len(index))}
x = pd.DataFrame(data, index=index)
x

,x
t,
2021-12-01 06:00:17,0
2021-12-01 06:00:18,1
2021-12-01 06:00:19,2
2021-12-01 06:00:20,3
2021-12-01 06:00:21,4
...,...
2021-12-01 06:02:53,156
2021-12-01 06:02:54,157
2021-12-01 06:02:55,158


In this example the values sampled every minute should be the following:

In [3]:
print(x.loc[['2021-12-01 06:01:00', '2021-12-01 06:02:00']])

                       x
t                       
2021-12-01 06:01:00   43
2021-12-01 06:02:00  103


In [4]:
# First, lets try sampling the signal at 10 second intervals 
# and organize the samples into a dataframe so we can see them
resample_10sec = x.resample(rule='10s', offset=0)
keys = [k for k, v in resample_10sec]
samples = [v for k, v in resample_10sec]
summary = pd.concat([s.reset_index(drop=True) for s in samples], axis=1, keys=keys).T
summary.head()

,,0,1,2,3,4,5,6,7,8,9
2021-12-01 06:00:10,x,0.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-01 06:00:20,x,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
2021-12-01 06:00:30,x,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0
2021-12-01 06:00:40,x,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0
2021-12-01 06:00:50,x,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0


In [5]:
# Drop the unnecessary 'x' values in the time index
summary = summary.droplevel(1, axis=0)
summary.head()

,0,1,2,3,4,5,6,7,8,9
2021-12-01 06:00:10,0.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-01 06:00:20,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
2021-12-01 06:00:30,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0
2021-12-01 06:00:40,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0
2021-12-01 06:00:50,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0


In [6]:
# Note that each sample of 10 values starts at one sample time
# and ends just before the next. This is not what we want.

In [7]:
# To sample values that are centered around the desired
# sample time, we can use the offset argument
resample_10sec = x.resample(rule='10s', offset='5s')
keys = [k for k, v in resample_10sec]
samples = [v for k, v in resample_10sec]
summary = pd.concat([s.reset_index(drop=True) for s in samples], axis=1, keys=keys)
summary = summary.droplevel(1, axis=1).T
summary.head()

,0,1,2,3,4,5,6,7,8,9
2021-12-01 06:00:15,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,NaN,NaN
2021-12-01 06:00:25,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0
2021-12-01 06:00:35,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0
2021-12-01 06:00:45,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0
2021-12-01 06:00:55,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0


In [8]:
# Now, note that each sample starts 5 seconds before each 
# desired sample time (e.g. 06:00:25) and ends 4 seconds
# after (e.g. 06:00:34)

In [9]:
summary.tail()

,0,1,2,3,4,5,6,7,8,9
2021-12-01 06:02:15,118.0,119.0,120.0,121.0,122.0,123.0,124.0,125.0,126.0,127.0
2021-12-01 06:02:25,128.0,129.0,130.0,131.0,132.0,133.0,134.0,135.0,136.0,137.0
2021-12-01 06:02:35,138.0,139.0,140.0,141.0,142.0,143.0,144.0,145.0,146.0,147.0
2021-12-01 06:02:45,148.0,149.0,150.0,151.0,152.0,153.0,154.0,155.0,156.0,157.0
2021-12-01 06:02:55,158.0,159.0,160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Drop any incomplete samples (those with NaNs at the beginning
# and end of the signal data)
summary = summary.dropna()
summary

,0,1,2,3,4,5,6,7,8,9
2021-12-01 06:00:25,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0
2021-12-01 06:00:35,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0
2021-12-01 06:00:45,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0
2021-12-01 06:00:55,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0
2021-12-01 06:01:05,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0
2021-12-01 06:01:15,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0
2021-12-01 06:01:25,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0
2021-12-01 06:01:35,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0
2021-12-01 06:01:45,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0
2021-12-01 06:01:55,98.0,99.0,100.0,101.0,102.0,103.0,104.0,105.0,106.0,107.0


In [11]:
# Finally adjust the index values so that they
# represent the desired sample times
summary = summary.set_index(summary.index + pd.Timedelta('5s'))

In [12]:
# Now calculate the median values
x_sampled_10sec = summary.median(axis=1)
x_sampled_10sec

2021-12-01 06:00:30     12.5
2021-12-01 06:00:40     22.5
2021-12-01 06:00:50     32.5
2021-12-01 06:01:00     42.5
2021-12-01 06:01:10     52.5
2021-12-01 06:01:20     62.5
2021-12-01 06:01:30     72.5
2021-12-01 06:01:40     82.5
2021-12-01 06:01:50     92.5
2021-12-01 06:02:00    102.5
2021-12-01 06:02:10    112.5
2021-12-01 06:02:20    122.5
2021-12-01 06:02:30    132.5
2021-12-01 06:02:40    142.5
2021-12-01 06:02:50    152.5
dtype: float64

In [13]:
new_index_values = pd.date_range(
    x_sampled_10sec.first_valid_index().ceil('min'),
    x_sampled_10sec.last_valid_index().floor('min'),
    freq='1min'
)
new_index_values

DatetimeIndex(['2021-12-01 06:01:00', '2021-12-01 06:02:00'], dtype='datetime64[ns]', freq='T')

In [14]:
# Now we can resample this sequence at the lower frequency
# we are interested in
x_sampled_1min = x_sampled_10sec.loc[new_index_values]
x_sampled_1min

2021-12-01 06:01:00     42.5
2021-12-01 06:02:00    102.5
dtype: float64

***Note that the median values calculated are not quite the
same as the original sampled values at these times. This is
because this method uses a window that is an even-number of
samples in length, and the original value at exactly the 
desired time is considered to be a member of the second half 
of window, not the first.***

However, if you choose a window size that is an odd number
of samples (but still a factor of the desired sampling 
frequency) then this problem does not occur.

For example, if down sampling to a period of 1 minute, the
following half-window sizes will work:

```python
half_window_size=1.5  # 60 / 1.5 = 40.0, window_size = 3
half_window_size=2.5  # 60 / 2.5 = 24.0, window_size = 5
half_window_size=7.5  # 60 / 7.5 = 8.0, window_size = 15
```

In most cases though, (unless your data values are all unique
as in this case), the median will be equal to the most common 
value, or close to it.

In [15]:
# This can be done in just a few lines if you're not concerned
# about dropping the first and last sample which might not be
# a complete set of values
x_10sec_medians = x.resample(rule='10s', offset='5s').median()
x_10sec_medians = x_10sec_medians.set_index(x_10sec_medians.index + pd.Timedelta('5s'))
x_sampled_1min = x_10sec_medians.resample(rule='1min').agg('first')
x_sampled_1min

,x
t,
2021-12-01 06:00:00,3.5
2021-12-01 06:01:00,42.5
2021-12-01 06:02:00,102.5
2021-12-01 06:03:00,159.0


In [16]:
# Here is a function to do the whole operation

def downsample_using_local_median(x, freq, new_freq, 
                                  half_window_size=2.5):
    """Samples from a dataframe x of time series at a lower
    frequency by calculating the median value over a window
    of points around each desired sample time.
    
    For this method, the desired sample period must be 
    divisible by the half the window size. For example, if 
    the original data is sampled every second, and the 
    desired sampling period is every minute, then the half-
    window size could be 1.5, 2.5, 5, 7.5, or 10, for 
    example. Note that if the whole window size is an odd 
    number, then the window is more precicely centred
    around the desired sample-periods.
    
    Arguments
    ---------
    x : dataframe
        Pandas dataframe with a date-time index that has a
        constant sample period.
    freq : str
        Sample period of the signals in x expressed as a 
        Pandas frequency alias (e.g. 'min' or '15s').
    new_freq : str
        Desired sampling period after resampling, expressed
        as a Pandas frequency alias.
    half_window_size : float
        Number of samples in the calculation window 
        divided by two. E.g. for a window size of 5,
        half_window_size = 2.5. However, half_window_size
        must be a factor of the desired sampling period.
        E.g. 45 seconds is a factor of 6 minutes but not
        a factor of 5 minutes.
    """
    x = pd.DataFrame(x)
    # Check that half-window size is a factor of the
    # desired sampling period.
    freq = pd.Timedelta(freq)
    new_freq = pd.Timedelta(new_freq)
    offset = freq * half_window_size
    assert((new_freq / offset).is_integer())
    x_medians = x.resample(rule=offset*2, offset=offset).median()
    x_medians = x_medians.set_index(x_medians.index + offset)
    new_index = pd.date_range(
        x_medians.first_valid_index().ceil(new_freq),
        x_medians.last_valid_index().floor(new_freq),
        freq=new_freq
    )
    return x_medians.loc[new_index]


In [17]:
downsample_using_local_median(x, '1sec', '1min', half_window_size=5)

,x
t,
2021-12-01 06:01:00,42.5
2021-12-01 06:02:00,102.5
2021-12-01 06:03:00,159.0


In [18]:
# Sample using a window with an odd number of samples
downsample_using_local_median(x, '1sec', '1min', half_window_size=2.5)

,x
t,
2021-12-01 06:01:00,43.0
2021-12-01 06:02:00,103.0


## Test function

In [19]:
# Test data 1
y = pd.Series(
    [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    index=pd.date_range("2021-01-01 00:00:56", "2021-01-01 00:01:06", freq='S')
)
y

2021-01-01 00:00:56    100
2021-01-01 00:00:57    101
2021-01-01 00:00:58    102
2021-01-01 00:00:59    103
2021-01-01 00:01:00    104
2021-01-01 00:01:01    105
2021-01-01 00:01:02    106
2021-01-01 00:01:03    107
2021-01-01 00:01:04    108
2021-01-01 00:01:05    109
2021-01-01 00:01:06    110
Freq: S, dtype: int64

In [20]:
# Result should be one of these values
y_target = y.loc["2021-01-01 00:01:00"]
y_target_off = y_target - 0.5
y_target, y_target_off

(104, 103.5)

In [21]:
y_sampled = downsample_using_local_median(y, '1sec', '1min', half_window_size=1.5)
assert(y_sampled.values.item() == y_target)

In [22]:
y_sampled = downsample_using_local_median(y, '1sec', '1min', half_window_size=2.5)
assert(y_sampled.values.item() == y_target)

In [23]:
y_sampled = downsample_using_local_median(y, '1sec', '1min', half_window_size=3)
assert(y_sampled.values.item() == y_target_off)

In [24]:
# When window is too big returns median of all values
y_sampled = downsample_using_local_median(y, '1sec', '1min', half_window_size=7.5)
assert(y_sampled.values.item() == y.median())

In [25]:
# Check if y has nan values
y.loc['2021-01-01 00:01:02'] = np.nan
y

2021-01-01 00:00:56    100.0
2021-01-01 00:00:57    101.0
2021-01-01 00:00:58    102.0
2021-01-01 00:00:59    103.0
2021-01-01 00:01:00    104.0
2021-01-01 00:01:01    105.0
2021-01-01 00:01:02      NaN
2021-01-01 00:01:03    107.0
2021-01-01 00:01:04    108.0
2021-01-01 00:01:05    109.0
2021-01-01 00:01:06    110.0
Freq: S, dtype: float64

In [26]:
# Window of size 3 samples should not be affected
y_sampled = downsample_using_local_median(y, '1sec', '1min', half_window_size=1.5)
assert(y_sampled.values.item() == y_target)

In [27]:
# Window of size 5 samples
y_sampled = downsample_using_local_median(y, '1sec', '1min', half_window_size=2.5)
assert(y_sampled.values.item() == np.nanmedian([102, 103, 104, 105, np.nan]))